In [1]:
import sys
sys.path.append("/home/apbarret/src/SunlightUnderSeaIce/sunderseaice")  # Allow import of modules for sunderseaice
sys.path.append("/home/apbarret/src/nsidc_projections/nsidc_projections")

import os
import datetime as dt
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LinearRing, Polygon
from shapely.geometry.polygon import orient
from pyproj import Transformer

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

import h5py
import xarray as xr

import download.nsidc_download as cmr
from download.icesat2_search import read_polarstern_track, select_midday, POLARSTERN_TRACK_FILE
from grid import SSMI_PolarStereoNorth25km 

DATAPATH = '/home/apbarret/src/SunlightUnderSeaIce/data'

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    map_proj = SSMI_PolarStereoNorth25km.to_cartopy()
map_extent = [-2349878.8355433852, 2349878.8355433857, -2349878.8355433852, 2349878.8355433857]

In [2]:
from collections import namedtuple
MosaicFloe = namedtuple("FloeDates", ['start', 'end'])
floe1 = MosaicFloe(dt.datetime(2019, 10, 4), dt.datetime(2020, 5, 17))
floe2 = MosaicFloe(dt.datetime(2020, 6, 19), dt.datetime(2020, 7, 31))
floe3 = MosaicFloe(dt.datetime(2020, 8, 21), dt.datetime(2020, 9, 20))

In [3]:
transformer_fwd = Transformer.from_crs(4326, SSMI_PolarStereoNorth25km.epsg, always_xy=True)
transformer_inv = Transformer.from_crs(SSMI_PolarStereoNorth25km.epsg, 4326, always_xy=True)

def pandas_to_geopandas(df, crs="EPSG:4326"):
    """Returns a geopandas dataseries"""
    geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
    return gpd.GeoSeries(geometry, index=df.index, crs=crs)

def buffer_point(lon, lat, buffer_radius=20000., resolution=3):
    """Returns a correctly orientated buffer polygon around a point
    
    point - a tuple containing (longitude, latitude) pair"""
    projected_point = Point(transformer_fwd.transform(lon, lat))
    projected_poly = projected_point.buffer(buffer_radius, resolution)
    poly = Polygon(transformer_inv.itransform(projected_poly.exterior.coords))
    poly_ccw = orient(poly, sign=1.0)
    return poly


def shapely_to_cmr(poly):
    """Converts shapely polygon to string containing lon, lat
       expected by CMR.  CMR expects coordinates to be counter-
       clockwise.  Code automatically reverses this"""
    poly_ccw = orient(poly, sign=1.0)
    assert LinearRing(poly_ccw.exterior.coords).is_ccw, "Polygon is not counter-clockwise"
    return ",".join([f"{x},{y}" for x, y in poly_ccw.exterior.coords])


class Trajectory(pd.DataFrame):
    @property
    def _constructor(self):
        return Trajectory
    
    def midday(self):
        """Returns 1200 (midday) position""" 
        return self[(self.index.hour == 12) & (self.index.minute == 0)]
    
    def to_cmr_query(self):
        """Returns list of tuples containing date and buffer for cmr search"""
        pass
    
    def get_buffer(self):
        """Adds a buffer polygon object as a column"""
        self["buffer"] = [buffer_point(lon, lat) for lon, lat in zip(df.Longitude, df.Latitude)]

    def get_cmr_query(self):
        """Returns tuple of search date and polygon vertices as string"""
        return [(index, shapely_to_cmr(poly)) for index, poly in self.buffer.iteritems()]


In [4]:
df = read_polarstern_track()
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs="EPSG:4326")
gdf = gdf.to_crs(SSMI_PolarStereoNorth25km.epsg)  # Buffers need to be calculated in projected coordinates, convert to NSIDC PolarSterographc
gdf.loc[:, "geometry"] = gdf.geometry.buffer(20000., 3)  # Generate a buffer polygon
gdf = gdf.to_crs(4326)  #  Set back to lon-lat WGS84

gdf = gdf[floe2.start:floe2.end]  # Focus on floe2 
gdf_midday = gdf[(gdf.index.hour == 12) & (gdf.index.minute == 0)].copy()  # To avoid updating a view of gdf

gdf_midday.head()

,Latitude,Longitude,Speed,Course,geometry
Date,,,,,
2020-06-19 12:00:00,82.02769,8.74251,0.217,84.5,"POLYGON ((9.51163 81.87879, 8.88366 81.84501, ..."
2020-06-20 12:00:00,82.02692,9.15124,0.205,199.4,"POLYGON ((9.91271 81.87726, 9.28319 81.84411, ..."
2020-06-21 12:00:00,81.98196,9.18758,0.351,209.5,"POLYGON ((9.94418 81.83225, 9.31799 81.79915, ..."
2020-06-22 12:00:00,81.96777,9.32557,0.237,222.7,"POLYGON ((10.07830 81.81781, 9.45268 81.78492,..."
2020-06-23 12:00:00,81.99867,9.69658,0.223,192.6,"POLYGON ((10.44528 81.84802, 9.81585 81.81570,..."


In [5]:
shapely_to_cmr(gdf_midday.iloc[3,:].geometry)

'10.078302880815441,81.817809200106,10.512323997285574,81.89034790835252,10.636753149555467,81.9838464717769,10.408064655537508,82.07352694485158,9.878747549516635,82.13485973955459,9.1925222962977,82.15062045169547,8.544241862012756,82.11632240869798,8.11617137324634,82.04169749713458,8.020321719821991,81.94751873980789,8.27166097082281,81.8592435441352,8.795052842963516,81.79999741914631,9.452676702124208,81.78491763339973,10.078302880815441,81.817809200106'

In [6]:
product = 'ATL03'
version = '004'

assert gdf_midday.crs == "EPSG:4326", "Expects CRS is WGS84 (EPSG:4326), convert using to_crs(4326) method"

#big_list_of_granules = []
gdf_midday["granule"] = None
gdf_midday["metadata"] = None
for search_date, items in gdf_midday.iterrows():
    start_date, end_date = search_date - dt.timedelta(hours=12), search_date + dt.timedelta(hours=12)
    results = cmr.cmr_search(
        product, 
        version, 
        start_date.isoformat(),  # CMR API expects iso format date strings
        end_date.isoformat(), 
        polygon=shapely_to_cmr(items.geometry),
        verbose=False,
    )
    if not results:
        continue
    if len(results) > 2:
        raise ValueError("More than one value returned by search")
    if results[0].endswith('.h5'):
        gdf_midday.loc[search_date, "granule"] = results[0]
    else:
        raise ValueError(f"Expects first result to be path to HDF5 (.h5) got {results[0]}")
    if results[1].endswith('.xml'):
        gdf_midday.loc[search_date, "metadata"] = results[1]
    else:
        raise ValueError(f"Expects second result to be metadata (.xml) got {results[1]}")


In [7]:
gdf_midday = gdf_midday.dropna()
gdf_midday.to_csv("mosaic_polarstern_floe2_icesat2_granules.csv", columns=["Latitude", "Longitude", "granule", "metadata"])

In [12]:
!tail mosaic_polarstern_floe2_icesat2_granules.csv

2020-07-04 12:00:00,81.70845,6.70453,https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.004/2020.07.04/ATL03_20200704080254_01370804_004_01.h5,https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.004/2020.07.04/ATL03_20200704080254_01370804_004_01.iso.xml
2020-07-05 12:00:00,81.69782,6.24978,https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.004/2020.07.05/ATL03_20200705214551_01610804_004_01.h5,https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.004/2020.07.05/ATL03_20200705214551_01610804_004_01.iso.xml
2020-07-08 12:00:00,81.59656,3.25034,https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.004/2020.07.08/ATL03_20200708220310_02070804_004_01.h5,https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.004/2020.07.08/ATL03_20200708220310_02070804_004_01.iso.xml
2020-07-11 12:00:00,81.4458,1.53678,https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.004/2020.07.11/ATL03_20200711081154_02440804_004_01.h5,https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.004/2020.07.11/ATL03_20200711081154_02440804_004_01.iso.xml
2020-07-12 12:00:

## Download data using CMR

In [ ]:
!head /home/apbarret/src/SunlightUnderSeaIce/data/polarstern_track_full_cruise.txt

In [ ]:
gdf = pandas_to_geopandas(df)
gdf = gdf.to_crs(SSMI_PolarStereoNorth25km.epsg) 
gdf

In [ ]:
fig = plt.figure(figsize=(7,7))

ax = fig.add_subplot(projection=map_proj)
ax.set_extent(map_extent, crs=map_proj)
ax.add_feature(cfeature.LAND, facecolor='0.5')
gdf.plot(ax=ax, markersize=5)

In [ ]:
fig = plt.figure(figsize=(7,7))

ax = fig.add_subplot(projection=map_proj)
ax.set_extent(map_extent, crs=map_proj)
ax.add_feature(cfeature.LAND, facecolor='0.5')

gdf[start_leg01.strftime("%Y-%m-%d"):end_leg01.strftime("%Y-%m-%d")].plot(ax=ax, markersize=5, label="Floe 1")
gdf[start_leg02.strftime("%Y-%m-%d"):end_leg02.strftime("%Y-%m-%d")].plot(ax=ax, markersize=5, color='red', label="Floe 2")
gdf[start_leg03.strftime("%Y-%m-%d"):end_leg03.strftime("%Y-%m-%d")].plot(ax=ax, markersize=5, color='green', label="Floe 3")

In [ ]:
floe02 = gdf[start_leg02.strftime("%Y-%m-%d"):end_leg02.strftime("%Y-%m-%d")]
floe02_midday = floe02[(floe02.index.hour == 12) & (floe02.index.minute == 0)]
floe02_midday.head()

## Create buffers
Buffers have to be calculated in projected coordinates.  They are then converted to latitude-longitude in WGS84 (EPSG:4326)

In [ ]:
buffer_radius = 20000.  # in metres
polygon_resolution = 3
search_buffers = floe02_midday.buffer(buffer_radius, polygon_resolution).to_crs(4326)
search_buffers.head()

In [ ]:
product = 'ATL10'
version = '004'

assert search_buffers.crs == "EPSG:4326", "Expects CRS is WGS84 (EPSG:4326), convert using to_crs(4326) method"

big_list_of_granules = []
for search_date, polygon in search_buffers.iteritems():
    start_date, end_date = search_date - dt.timedelta(hours=12), search_date + dt.timedelta(hours=12)
    results = cmr.cmr_search(
        product, 
        version, 
        start_date.isoformat(),  # CMR API expects iso format date strings
        end_date.isoformat(), 
        polygon=shapely_to_cmr(polygon),
        )
    big_list_of_granules = big_list_of_granules + results
    
for g in big_list_of_granules:
    print(g)

In [ ]:
cmr.cmr_download(big_list_of_granules, outpath='/media/apbarret/andypbarrett_work/Data/ICESat-2/ATL10')

In [ ]:
ls -ltr /media/apbarret/andypbarrett_work/Data/ICESat-2/ATL10 | tail

In [ ]:
from readers import icesat2

In [ ]:
ICESAT2_PATH = Path("/media/apbarret/andypbarrett_work/Data/ICESat-2/ATL10")
h5files = ICESAT2_PATH.glob("ATL10-01_20200[67]*.h5")
tlat = []
tlon = []
name = []
for f in h5files:
    ds = icesat2.atl10(f)
    ds = ds.where(ds.freeboard < 3.e38).dropna(dim='x')
    name.append(f.name)
    tlat.append(ds.latitude.values)
    tlon.append(ds.longitude.values)

In [ ]:
x0, y0 = map_proj.transform_point(-34., 87., ccrs.PlateCarree())
x1, y1 = map_proj.transform_point(7., 73., ccrs.PlateCarree())
[x0, x1, y0, y1]

In [ ]:
floe2_extent = [x0, x1, y0, y1]

fig = plt.figure(figsize=(10,7))

ax = fig.add_subplot(projection=map_proj)
ax.set_extent(floe2_extent, crs=map_proj)
#ax.set_extent(map_extent, crs=map_proj)
ax.add_feature(cfeature.LAND, facecolor='0.5')
ax.gridlines()

gdf[start_leg02.strftime("%Y-%m-%d"):end_leg02.strftime("%Y-%m-%d")].plot(ax=ax, markersize=5, color='red', label="Polarstern")

for xlat, xlon in zip(tlat, tlon):
    pts = map_proj.transform_points(ccrs.PlateCarree(), xlon[::1000], xlat[::1000])
    x = pts[:, 0]
    y = pts[:, 1]
    ax.plot(x, y, 'k-', transform=map_proj)

ax.set_title("ICESat-2 tracks coincident with Polarstern Drift")

#ax.plot(tlon[id][::1000], tlat[id][::1000], 'k-', transform=ccrs.PlateCarree())
ax.legend()

fig.savefig("icesat2_tracks_with_polarstern_floe2.png")

In [ ]:
for xlat, xlon in zip(tlat, tlon):
    print(xlat.min(), xlat.max())
    print(xlon.min(), xlon.max())